In [1]:
!pip install --upgrade gspread gspread-dataframe google-auth python-docx pillow pandas PyPDF2 \
google-api-python-client lxml tqdm docxcompose -q
print("Bibliotecas instaladas/actualizadas")

# === Sistema y utilidades ===
import os
import re
import shutil
import tempfile
import threading
import concurrent.futures
import unicodedata
from datetime import datetime

# === Entrada/Salida en memoria ===
import io
from io import BytesIO

# === HTTP y web ===
import requests
import urllib.parse

# === Datos ===
import pandas as pd
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from tqdm.notebook import tqdm

# === Google APIs ===
from google.colab import auth
from google.auth import default
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseUpload, MediaFileUpload
from googleapiclient.errors import HttpError

# === Documentos Word ===
from docx import Document
from docx.shared import Pt, Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docxcompose.composer import Composer

# === PDF ===
from PyPDF2 import PdfMerger

# === Imágenes ===
from PIL import Image

# Autenticación
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Montar Drive
from google.colab import drive
drive.mount('/content/drive')
print("Autenticación completada")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 929.1 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.3/221.3 kB 6.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.38.0, but you have google-auth 2.41.1 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.
gradio 5.49.1 requires pillow<12.0

In [10]:
#SE TENDRÁ QUE GUARDAR MANUALMENTE LA VERSIÓN DEL SHEETS AL EXCEL CREADO EN LA CREACIÓN DE RECOPILADORES MIENTRAS NO SE CAMBIE EL SCRIPT ANTERIOR, PORQUE
#EL ID QUE SE USA PARA EL SPREADHEET_ID TIENE QUE SER EL DE UN SHEETS

#ASIMISMO, EN EL SHEETS/XLSX CREADO, SE DEBE AÑADIR LAS COLUMNAS QUE CONTIENEN LOS DATOS QUE ENTRAN A LA FICHA

# ==========================
# Configuración global - MODIFICADO PARA SURVEYMONKEY
# ==========================

# ID del Google Sheets con los enlaces de SurveyMonkey
SPREADSHEET_ID = '1HJvEBWT4eptbOVc0S_gtHM3ADJF5y4PDNd5GNzq70ig'  # Reemplazar ID con el Sheets creado en la creación de recopiladores
DRIVE_BASE_PATH = '/content/drive/MyDrive/ENCUESTAS_SURVEYMONKEY'  # Carpeta principal
MAX_WORKERS = 8

# --- Caché para servicios y carpetas ---
folder_cache = {}
service_cache = threading.local()

# ==========================
# Helpers de servicio y Drive (MANTENIDOS)
# ==========================
def get_drive_service():
    if not hasattr(service_cache, "drive_service"):
        service_cache.drive_service = build('drive', 'v3', credentials=creds)
    return service_cache.drive_service

def get_drive_folder_id(folder_path):
    if folder_path in folder_cache:
        return folder_cache[folder_path]

    service = get_drive_service()

    if folder_path.startswith('/content/drive/MyDrive'):
        base_path = folder_path.replace('/content/drive/MyDrive', '', 1).strip('/')
    else:
        base_path = folder_path.strip('/')

    if not base_path:
        return 'root'

    parts = base_path.split('/')
    current_id = 'root'

    for part in parts:
        if not part:
            continue

        query = (
            f"name='{part}' and mimeType='application/vnd.google-apps.folder' "
            f"and '{current_id}' in parents and trashed=false"
        )
        results = service.files().list(q=query, fields="files(id)").execute()
        items = results.get('files', [])

        if items:
            current_id = items[0]['id']
        else:
            folder_metadata = {
                'name': part,
                'mimeType': 'application/vnd.google-apps.folder',
                'parents': [current_id]
            }
            folder = service.files().create(body=folder_metadata, fields='id').execute()
            current_id = folder['id']

    folder_cache[folder_path] = current_id
    return current_id

def get_file_id_in_folder(service, folder_id, file_name):
    query = f"name='{file_name}' and '{folder_id}' in parents and trashed=false"
    try:
        results = service.files().list(q=query, fields="files(id)").execute()
        items = results.get('files', [])
        return items[0]['id'] if items else None
    except HttpError:
        return None

# ==========================
# Helpers de formato / nombres (MODIFICADOS)
# ==========================
def agregar_texto_formateado(paragraph, texto, bold=True, italic=False, font_size=11,
                             alignment=WD_ALIGN_PARAGRAPH.JUSTIFY):
    """Añade texto al párrafo con estilo consistente."""
    run = paragraph.add_run(str(texto))
    run.bold = bold
    run.italic = italic
    run.font.size = Pt(font_size)
    run.font.name = 'Arial'
    paragraph.alignment = alignment
    pf = paragraph.paragraph_format
    pf.line_spacing = 1.0
    pf.space_after = Pt(4)
    return paragraph

def slugify_filename(name: str) -> str:
    """Sanear nombres de archivo para evitar caracteres problemáticos."""
    name = name.replace('/', '_').replace('\\', '_').strip()
    name = re.sub(r'\s+', ' ', name).strip()
    name = re.sub(r'[^A-Za-z0-9._\-\s]+', '_', name)
    if len(name) > 150:
        base, ext = os.path.splitext(name)
        name = base[:140] + ext
    return name

# ==========================
# Mapeo de columnas para SurveyMonkey (NUEVO)
# ==========================
COLUMNAS_SURVEYMONKEY = {
    "id_registro": "CURSO-HORARIO",          # CURSO-HORARIO del Excel
    "url": "ENLACE_ENCUESTA",                # URL de SurveyMonkey
    "salon": "PABELLON",
    "nombre_curso": "CURSO",
    "horario_curso": "HORARIO",
    "docente": "DOCENTE",
    "total_alumnos": "ALUMNOS"
}

def get_val(row, clave, default=""):
    col = COLUMNAS_SURVEYMONKEY.get(clave, clave)
    val = row.get(col, default)
    return "" if val is None else str(val).strip()

# ==========================
# Lectura de datos desde Google Sheets (MODIFICADO)
# ==========================
print("Leyendo datos de Google Sheets...")

# Abrir el Google Sheets
spreadsheet = gc.open_by_key(SPREADSHEET_ID)

# Obtener todas las hojas disponibles
worksheets = spreadsheet.worksheets()
print("Hojas disponibles:")
for ws in worksheets:
    print(f"  - {ws.title}")

# Leer la hoja que contiene los datos (ajusta el nombre según tu Sheets)
try:
    # Intenta leer la hoja que contiene los datos de SurveyMonkey
    worksheet = spreadsheet.worksheet('Hoja 1')  # Cambia por el nombre correcto
    df = pd.DataFrame(worksheet.get_all_records())
    print(f"Datos leídos: {len(df)} registros")

    # Mostrar las columnas disponibles
    print("Columnas disponibles:")
    for col in df.columns:
        print(f"  - {col}")

except Exception as e:
    print(f"Error al leer la hoja: {e}")
    # Si falla, intenta con la primera hoja
    worksheet = worksheets[0]
    df = pd.DataFrame(worksheet.get_all_records())
    print(f"Datos leídos de la primera hoja: {len(df)} registros")

# Mostrar una muestra de los datos
print("\nMuestra de datos:")
print(df.head())

Leyendo datos de Google Sheets...
Hojas disponibles:
  - Enlaces_Encuestas
Error al leer la hoja: Hoja 1
Datos leídos de la primera hoja: 6 registros

Muestra de datos:
  CURSO-HORARIO                        ENLACE_ENCUESTA  ID_COLECTOR  \
0   DEM223-0301  https://es.surveymonkey.com/r/2JJK65F    462938313   
1   DEP217-0304  https://es.surveymonkey.com/r/2JJCPWS    462938315   
2   DEC286-0503  https://es.surveymonkey.com/r/2JJRCNP    462938316   
3   DEC287-0605  https://es.surveymonkey.com/r/2JJNB6Y    462938317   
4   DEE330-0602  https://es.surveymonkey.com/r/2JJH2VN    462938318   

        FECHA_CREACION  ESTADO    PABELLON    CURSO     HORARIO  \
0  2025-11-17 22:16:46  ACTIVO  Z313\nP201  DERECHO  Presencial   
1  2025-11-17 22:16:46  ACTIVO        D106  DERECHO  Presencial   
2  2025-11-17 22:16:46  ACTIVO  D205\nD106  DERECHO  Presencial   
3  2025-11-17 22:16:46  ACTIVO        H214  DERECHO  Presencial   
4  2025-11-17 22:16:46  ACTIVO  D102\nH212  DERECHO  Presencial   

 

In [11]:
# Función para procesar el horario y dejar solo el día y hora
def procesar_horario(horario):
    """
    Extrae solo el día y la hora del horario dado, eliminando lo que viene después de la hora (como aula, 'C', 'CP').
    Ejemplo:
      "LUN 10:30-12:30 C H213" -> "LUN 10:30-12:30"
      "MIE 07:00-09:00 CP H112" -> "MIE 07:00-09:00"
      "LUN 10:30-12:30 C H213<br>MIE 07:00-09:00 CP H112" -> "LUN 10:30-12:30 / MIE 07:00-09:00"
    """
    if not horario or pd.isna(horario):
        return ""

    # Si hay saltos de línea o <br>, separar en múltiples horarios
    horarios = str(horario).replace('<br>', '\n').split('\n')
    horarios_procesados = []

    for h in horarios:
        h = h.strip()
        if not h:
            continue

        # Patrón para capturar solo el día y la hora
        patron = r"([A-Z]{3})\s+(\d{2}:\d{2}-\d{2}:\d{2})"
        matches = re.findall(patron, h)

        for match in matches:
            # Concatenamos solo el día y la hora
            horarios_procesados.append(f"{match[0]} {match[1]}")

    # Unimos los horarios en un solo string, separados por " / "
    return " / ".join(horarios_procesados) if horarios_procesados else str(horario)

# Función para generar PDF desde el Word
def generar_pdf_desde_word(service, word_file_id, id_registro, current_seleccion):
    """Convierte un archivo Word a PDF usando Google Drive"""
    temp_doc_id = None
    try:
        # Crear un documento temporal de Google Docs para la conversión
        temp_doc_name = f'TEMP_PDF_CONV_{id_registro}_{os.urandom(4).hex()}'
        temp_doc = service.files().copy(
            fileId=word_file_id,
            body={'name': temp_doc_name, 'mimeType': 'application/vnd.google-apps.document'}
        ).execute()
        temp_doc_id = temp_doc.get('id')

        # Exportar a PDF
        pdf_content = service.files().export_media(fileId=temp_doc_id, mimeType='application/pdf').execute()

        # Subir el PDF a Drive
        pdf_folder_id = get_drive_folder_id(f'{DRIVE_BASE_PATH}/Fichas_pdf/{current_seleccion}')
        pdf_file_name = f'{id_registro}.pdf'

        # Verificar si el PDF ya existe
        existing_pdf_id = get_file_id_in_folder(service, pdf_folder_id, pdf_file_name)
        pdf_media = MediaIoBaseUpload(BytesIO(pdf_content), mimetype='application/pdf')

        if existing_pdf_id:
            service.files().update(fileId=existing_pdf_id, media_body=pdf_media).execute()
        else:
            service.files().create(body={'name': pdf_file_name, 'parents': [pdf_folder_id]}, media_body=pdf_media).execute()

        pdf_file_path = f'{DRIVE_BASE_PATH}/Fichas_pdf/{current_seleccion}/{pdf_file_name}'
        print(f'  ✓ PDF generado: {id_registro}')
        return pdf_file_path

    except Exception as e:
        print(f"  ERROR generando PDF {id_registro}: {str(e)[:200]}")
        return None
    finally:
        # Limpiar documento temporal
        try:
            if temp_doc_id:
                service.files().delete(fileId=temp_doc_id).execute()
        except Exception:
            pass


In [12]:
# Función principal de generación de fichas para SurveyMonkey INCLUYENDO QR
def generar_ficha(row_data, current_seleccion="General", qr_image_bytes=None):
    """
    Genera una ficha con la estructura solicitada para SurveyMonkey INCLUYENDO QR.
    Retorna una tupla (ruta_word, ruta_pdf)
    """
    service = get_drive_service()

    # Obtener los valores de las columnas del Google Sheets
    id_registro = slugify_filename(get_val(row_data, "id_registro", "N_A"))
    nombre_curso = get_val(row_data, "nombre_curso", "")
    docente = get_val(row_data, "docente", "Docente no especificado")
    total_alumnos = get_val(row_data, "total_alumnos", "0")
    horario = get_val(row_data, "horario_curso", "")  # El horario a procesar
    pabellon = get_val(row_data, "salon", "")  # El pabellón a procesar
    url = get_val(row_data, "url", "")  # URL para el QR

    # Procesar el horario y pabellón
    horario_procesado = procesar_horario(horario)
    pabellon_procesado = procesar_pabellones(pabellon)

    print(f"Procesando ficha: {id_registro}")

    # Buffers y temporales
    doc_buffer = None
    word_file_path = None
    pdf_file_path = None

    try:
        # Crear el documento Word con la ficha - Márgenes restaurados
        doc = Document()
        section = doc.sections[0]
        section.page_width = Inches(8.27)   # A4
        section.page_height = Inches(11.69)
        # Márgenes más amplios para centrado
        section.top_margin = Inches(0.5)
        section.bottom_margin = Inches(0.5)
        section.left_margin = Inches(0.7)
        section.right_margin = Inches(0.7)

        # INCLUIR QR EN LA FICHA (si está disponible)
        if qr_image_bytes:
            try:
                qr_para = doc.add_paragraph()
                qr_para.alignment = WD_ALIGN_PARAGRAPH.CENTER
                qr_image_bytes.seek(0)  # Resetear el buffer
                qr_para.add_run().add_picture(qr_image_bytes, width=Pt(400))  # Tamaño del QR
                # Eliminar espacio después del QR
                doc.add_paragraph().paragraph_format.space_after = Pt(0)
                print(f"  ✓ QR incluido en la ficha")
            except Exception as e:
                print(f"  ⚠️ Error incluyendo QR en ficha: {str(e)[:100]}")
        else:
            print(f"  ⚠️ No hay QR para incluir en la ficha")

        # PABELLÓN - Formato mejorado
        pabellon_para = doc.add_paragraph()
        pabellon_run = pabellon_para.add_run(f"FACULTAD: DERECHO                   PABELLÓN Y AULA: {pabellon_procesado}")
        pabellon_run.font.size = Pt(10)
        pabellon_run.font.name = 'Arial'
        pabellon_run.bold = True
        pabellon_para.paragraph_format.space_after = Pt(6)

        # NOMBRE DEL CURSO
        if nombre_curso:
            curso_para = doc.add_paragraph()
            curso_run = curso_para.add_run(f"CURSO: {nombre_curso}")
            curso_run.font.size = Pt(10)
            curso_run.font.name = 'Arial'
            curso_run.bold = True
            curso_para.paragraph_format.space_after = Pt(6)

        # HORARIO
        horario_para = doc.add_paragraph()
        horario_run = horario_para.add_run(f"HORARIO DEL CURSO: {horario_procesado}")
        horario_run.font.size = Pt(10)
        horario_run.font.name = 'Arial'
        horario_run.bold = True
        horario_para.paragraph_format.space_after = Pt(6)

        # HORARIO-CURSO (NUEVO CAMPO)
        horario_curso_para = doc.add_paragraph()
        horario_curso_run = horario_curso_para.add_run(f"HORARIO-CURSO: {id_registro}")
        horario_curso_run.font.size = Pt(10)
        horario_curso_run.font.name = 'Arial'
        horario_curso_run.bold = True
        horario_curso_para.paragraph_format.space_after = Pt(6)

        # DOCENTE
        docente_para = doc.add_paragraph()
        docente_run = docente_para.add_run(f"DOCENTE: {docente}")
        docente_run.font.size = Pt(10)
        docente_run.font.name = 'Arial'
        docente_run.bold = True
        docente_para.paragraph_format.space_after = Pt(8)

        # Total de alumnos y alumnos en aula en la misma línea
        alumnos_para = doc.add_paragraph()
        alumnos_run = alumnos_para.add_run(f"N° TOTAL DE ALUMNOS: {total_alumnos}               N° DE ALUMNOS EN AULA: ________")
        alumnos_run.font.size = Pt(10)
        alumnos_run.font.name = 'Arial'
        alumnos_run.bold = True
        alumnos_para.paragraph_format.space_after = Pt(6)

        # Rechazos
        rechazos_para = doc.add_paragraph()
        rechazos_run = rechazos_para.add_run("RECHAZOS: _________")
        rechazos_run.font.size = Pt(10)
        rechazos_run.font.name = 'Arial'
        rechazos_run.bold = True
        rechazos_para.paragraph_format.space_after = Pt(6)

        # APLICADOR/A
        aplicador_para = doc.add_paragraph()
        aplicador_run = aplicador_para.add_run("APLICADOR/A: _____________________________________________________________")
        aplicador_run.font.size = Pt(10)
        aplicador_run.font.name = 'Arial'
        aplicador_run.bold = True
        aplicador_para.paragraph_format.space_after = Pt(4)

        # FECHA y HORA en la misma línea
        fecha_hora_para = doc.add_paragraph()
        fecha_hora_run = fecha_hora_para.add_run("FECHA: __________________________       HORA DE APLICACIÓN: ________________________")
        fecha_hora_run.font.size = Pt(10)
        fecha_hora_run.font.name = 'Arial'
        fecha_hora_run.bold = True
        fecha_hora_para.paragraph_format.space_after = Pt(12)  # Aumento del espacio aquí

        # Sección de observaciones
        observaciones_titulo_para = doc.add_paragraph()
        observaciones_titulo_run = observaciones_titulo_para.add_run("OBSERVACIONES:")
        observaciones_titulo_run.font.size = Pt(10)
        observaciones_titulo_run.font.name = 'Arial'
        observaciones_titulo_run.bold = True
        observaciones_titulo_para.paragraph_format.space_after = Pt(6)  # Menos espacio entre título y observaciones

        # Espacio vacío para observaciones (múltiples líneas)
        for i in range(3):
            obs_para = doc.add_paragraph()
            obs_run = obs_para.add_run("________________________________________________________________________________")
            obs_run.font.size = Pt(10)
            obs_run.font.name = 'Arial'
            obs_para.paragraph_format.space_after = Pt(3)

        # URL DIRECTA (como respaldo) - CENTRADO CORRECTAMENTE
        if url:
            # Espacio antes del enlace
            doc.add_paragraph().paragraph_format.space_after = Pt(6)

            # Línea separadora centrada
            separador_para = doc.add_paragraph()
            separador_para.alignment = WD_ALIGN_PARAGRAPH.CENTER
            separador_run = separador_para.add_run("_________________________________")
            separador_run.font.size = Pt(10)
            separador_run.font.name = 'Arial'
            separador_para.paragraph_format.space_after = Pt(4)

            # URL centrada
            url_para = doc.add_paragraph()
            url_para.alignment = WD_ALIGN_PARAGRAPH.CENTER
            url_run = url_para.add_run(url)
            url_run.bold = True
            url_run.font.size = Pt(9)
            url_run.font.name = 'Arial'
            url_para.paragraph_format.space_after = Pt(6)

        # Buffer del DOCX
        doc_buffer = BytesIO()
        doc.save(doc_buffer)
        doc_buffer.seek(0)

        # Guardar la ficha en Google Drive (subir el archivo)
        word_file_name = f'{id_registro}.docx'
        word_folder_id = get_drive_folder_id(f'{DRIVE_BASE_PATH}/Fichas_word/{current_seleccion}')

        # Verificar si el archivo ya existe
        existing_word_id = get_file_id_in_folder(service, word_folder_id, word_file_name)
        word_media = MediaIoBaseUpload(doc_buffer, mimetype='application/vnd.openxmlformats-officedocument.wordprocessingml.document')

        if existing_word_id:
            service.files().update(fileId=existing_word_id, media_body=word_media).execute()
            word_file_id = existing_word_id
        else:
            file = service.files().create(body={'name': word_file_name, 'parents': [word_folder_id]}, media_body=word_media).execute()
            word_file_id = file.get('id')

        word_file_path = f'{DRIVE_BASE_PATH}/Fichas_word/{current_seleccion}/{word_file_name}'
        print(f'✔️ Ficha Word generada: {id_registro}')

        # Generar PDF también
        pdf_file_path = generar_pdf_desde_word(service, word_file_id, id_registro, current_seleccion)

        # Retornar la ruta del Word y PDF
        return word_file_path, pdf_file_path

    except Exception as e:
        print(f"  ERROR procesando {id_registro}: {str(e)[:200]}")
        return None, None

    finally:
        try:
            if doc_buffer:
                doc_buffer.close()
        except Exception:
            pass


In [13]:
# Función para generar QR y ficha con datos y subirlos a Drive
def generar_qr_y_ficha(row_data, current_seleccion="General"):
    """
    Genera un QR, ficha y las sube a Google Drive
    Retorna una tupla (ruta_word, ruta_pdf)
    """
    service = get_drive_service()

    # Obtener los valores necesarios
    id_registro = slugify_filename(get_val(row_data, "id_registro", "N_A"))
    url = get_val(row_data, "url", "")
    # ELIMINADO: curso = get_val(row_data, "curso", "Curso no especificado") - Ya no se usa

    print(f"Procesando QR y ficha: {id_registro}")

    # Buffers y temporales
    qr_buffer = None

    try:
        # 1. Generar QR si hay URL
        if url and url.startswith('http'):
            encoded_url = urllib.parse.quote(url)
            resp = requests.get(f'https://quickchart.io/qr?text={encoded_url}&size=500', timeout=15)

            if resp.status_code == 200:
                image = Image.open(BytesIO(resp.content))
                width, height = image.size
                crop_percentage = 0.09
                cropped_image = image.crop((
                    int(width * crop_percentage),
                    int(height * crop_percentage),
                    width - int(width * crop_percentage),
                    height - int(height * crop_percentage)
                ))

                qr_buffer = BytesIO()
                cropped_image.save(qr_buffer, format='PNG')
                qr_buffer.seek(0)

                # Subir QR a Drive
                qr_folder_id = get_drive_folder_id(f'{DRIVE_BASE_PATH}/QRs/{current_seleccion}')
                qr_file_name = f'{id_registro}.png'

                existing_qr_id = get_file_id_in_folder(service, qr_folder_id, qr_file_name)
                qr_media = MediaIoBaseUpload(qr_buffer, mimetype='image/png')

                if existing_qr_id:
                    service.files().update(fileId=existing_qr_id, media_body=qr_media).execute()
                else:
                    service.files().create(body={'name': qr_file_name, 'parents': [qr_folder_id]}, media_body=qr_media).execute()

                print(f'  ✓ QR generado y subido: {id_registro}')
            else:
                print(f'  ⚠️ Error generando QR: HTTP {resp.status_code}')
                qr_buffer = None
        else:
            print(f'  ⚠️ URL no disponible para QR: {url}')
            qr_buffer = None

        # 2. Generar ficha (Word y PDF) PASANDO EL QR
        # Hacer una copia del buffer QR para usar en la ficha
        qr_for_ficha = None
        if qr_buffer:
            qr_for_ficha = BytesIO(qr_buffer.getvalue())
            qr_for_ficha.seek(0)

        word_path, pdf_path = generar_ficha(row_data, current_seleccion, qr_image_bytes=qr_for_ficha)

        return word_path, pdf_path

    except Exception as e:
        print(f"  ERROR procesando QR y ficha {id_registro}: {str(e)[:200]}")
        return None, None

    finally:
        try:
            if qr_buffer:
                qr_buffer.close()
        except Exception:
            pass

In [14]:
# Función para procesar los pabellones y concatenarlos
def procesar_pabellones(pabellon):
    """
    Si hay más de un pabellón en la celda, los concatena con "/".
    También maneja casos con <br> o saltos de línea.
    Ejemplo:
      "H-213 H-214" -> "H-213/H-214"
      "D-103<br>D-206" -> "D-103/D-206"
      "H-211" -> "H-211" (sin cambios si solo hay un pabellón)
    """
    if not pabellon or pd.isna(pabellon):
        return ""

    # Si hay saltos de línea o <br>, separar en múltiples pabellones
    pabellones = str(pabellon).replace('<br>', '\n').split('\n')
    pabellones_limpios = []

    for p in pabellones:
        p = p.strip()
        if not p:
            continue

        # Eliminar "C" o "CP" si están presentes
        p = re.sub(r'\b(C|CP)\b', '', p).strip()

        # Eliminar espacios extra y agregar a la lista
        if p:
            pabellones_limpios.append(p)

    # Si hay más de un pabellón, los concatenamos con "/"
    if len(pabellones_limpios) > 1:
        return "/".join(pabellones_limpios)
    elif pabellones_limpios:
        return pabellones_limpios[0]
    else:
        return str(pabellon)

In [21]:
# ==========================
# Procesamiento principal (MODIFICADO)
# ==========================
print("Iniciando procesamiento de datos de SurveyMonkey...")

# Pre-crear estructura de carpetas
print("Creando estructura de carpetas...")
get_drive_folder_id(f"{DRIVE_BASE_PATH}/QRs/General")
get_drive_folder_id(f"{DRIVE_BASE_PATH}/Fichas_word/General")
get_drive_folder_id(f"{DRIVE_BASE_PATH}/Fichas_pdf/General")

# Verificar datos antes de procesar
print(f"Total de registros: {len(df)}")
print(f"Registros con URL válida: {df['ENLACE_ENCUESTA'].notna().sum()}")

# Procesamiento concurrente
print(f"Iniciando procesamiento con {MAX_WORKERS} hilos...")
all_word_files, all_pdf_files = [], []

futures = []
with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    for _, row in df.iterrows():
        # Verificar que la URL no esté vacía antes de enviar a procesar
        url_val = get_val(row, "url", "")
        if url_val and url_val != "https://es.surveymonkey.com/r/":
            futures.append(executor.submit(generar_qr_y_ficha, row, "General"))
        else:
            print(f"  SKIP: URL vacía o inválida para {get_val(row, 'id_registro', 'N/A')}")

    for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc="Generando archivos"):
        try:
            word_path, pdf_path = future.result()
            if word_path:
                all_word_files.append(word_path)
            if pdf_path:
                all_pdf_files.append(pdf_path)
        except Exception as e:
            print(f"Error procesando futuro: {e}")

print(f"Procesamiento completo: {len(all_word_files)} Word, {len(all_pdf_files)} PDF")

Iniciando procesamiento de datos de SurveyMonkey...
Creando estructura de carpetas...
Total de registros: 6
Registros con URL válida: 6
Iniciando procesamiento con 8 hilos...
Procesando QR y ficha: DEP217-0304
Procesando QR y ficha: DEM223-0301
Procesando QR y ficha: DEC286-0503
Procesando QR y ficha: DEC287-0605
Procesando QR y ficha: DEE330-0602
Procesando QR y ficha: DEE330-0605


Generando archivos:   0%|          | 0/6 [00:00<?, ?it/s]

  ✓ QR generado y subido: DEE330-0605
Procesando ficha: DEE330-0605
  ✓ QR incluido en la ficha
  ✓ QR generado y subido: DEP217-0304
Procesando ficha: DEP217-0304
  ✓ QR generado y subido: DEM223-0301
Procesando ficha: DEM223-0301
  ✓ QR incluido en la ficha
  ✓ QR incluido en la ficha
  ✓ QR generado y subido: DEC287-0605
Procesando ficha: DEC287-0605
  ✓ QR incluido en la ficha
  ✓ QR generado y subido: DEE330-0602
Procesando ficha: DEE330-0602
  ✓ QR incluido en la ficha
  ✓ QR generado y subido: DEC286-0503
Procesando ficha: DEC286-0503
  ✓ QR incluido en la ficha
✔️ Ficha Word generada: DEM223-0301
✔️ Ficha Word generada: DEE330-0605
✔️ Ficha Word generada: DEP217-0304
✔️ Ficha Word generada: DEE330-0602
✔️ Ficha Word generada: DEC287-0605
✔️ Ficha Word generada: DEC286-0503
  ✓ PDF generado: DEP217-0304
  ✓ PDF generado: DEC287-0605
  ✓ PDF generado: DEE330-0605
  ✓ PDF generado: DEC286-0503
  ✓ PDF generado: DEM223-0301
  ✓ PDF generado: DEE330-0602
Procesamiento completo: 6 Wo

In [22]:
############################################
#BLOQUE 5: Generación de Enlaces Compartibles
#############################################
import time
import random

def get_shareable_links():
    """Obtiene enlaces compartibles para SurveyMonkey"""
    service = get_drive_service()
    links = {}
    lock = threading.Lock()

    def process_folder(folder_path, link_type):
        nonlocal links
        try:
            folder_id = get_drive_folder_id(folder_path)
            if not folder_id:
                print(f"Carpeta no encontrada: {folder_path}")
                return

            print(f"Procesando: {folder_path} ({link_type})")

            page_token = None
            processed_files = 0

            while True:
                for attempt in range(3):
                    try:
                        query = f"'{folder_id}' in parents and trashed=false"
                        results = service.files().list(
                            q=query,
                            fields="nextPageToken, files(id, name, webViewLink)",
                            pageSize=400,
                            pageToken=page_token
                        ).execute()
                        break
                    except (HttpError, TimeoutError, ConnectionError) as e:
                        if attempt < 2:
                            wait_time = (2 ** attempt) + random.uniform(0.5, 1.5)
                            print(f"Reintento {attempt+1} para {folder_path} en {wait_time:.1f}s")
                            time.sleep(wait_time)
                        else:
                            print(f"Error permanente en {folder_path}: {str(e)[:100]}")
                            return

                files = results.get('files', [])
                processed_files += len(files)

                with lock:
                    for file in files:
                        base_name = file['name'].rsplit('.', 1)[0]
                        if base_name not in links:
                            links[base_name] = {}
                        links[base_name][link_type] = file['webViewLink']

                page_token = results.get('nextPageToken', None)
                if not page_token:
                    break

            print(f"{folder_path}: {processed_files} archivos procesados")

        except Exception as e:
            print(f"Error en {folder_path}: {str(e)[:200]}")

    folders_to_process = [
        (f"{DRIVE_BASE_PATH}/QRs/General", 'QR'),
        (f"{DRIVE_BASE_PATH}/Fichas_word/General", 'Word'),
        (f"{DRIVE_BASE_PATH}/Fichas_pdf/General", 'PDF')
    ]

    print(f"Iniciando procesamiento de {len(folders_to_process)} carpetas\n")
    for folder_path, link_type in folders_to_process:
        process_folder(folder_path, link_type)

    return pd.DataFrame.from_dict(links, orient='index')

print("Obteniendo enlaces compartibles...")
links_df = get_shareable_links()

# Rellenar valores faltantes
for col in ['QR', 'Word', 'PDF']:
    if col not in links_df.columns:
        links_df[col] = None

print(f"\nEnlaces obtenidos: {len(links_df)} registros")

Obteniendo enlaces compartibles...
Iniciando procesamiento de 3 carpetas

Procesando: /content/drive/MyDrive/ENCUESTAS_SURVEYMONKEY/QRs/General (QR)
/content/drive/MyDrive/ENCUESTAS_SURVEYMONKEY/QRs/General: 6 archivos procesados
Procesando: /content/drive/MyDrive/ENCUESTAS_SURVEYMONKEY/Fichas_word/General (Word)
/content/drive/MyDrive/ENCUESTAS_SURVEYMONKEY/Fichas_word/General: 6 archivos procesados
Procesando: /content/drive/MyDrive/ENCUESTAS_SURVEYMONKEY/Fichas_pdf/General (PDF)
/content/drive/MyDrive/ENCUESTAS_SURVEYMONKEY/Fichas_pdf/General: 6 archivos procesados

Enlaces obtenidos: 6 registros


In [23]:
######################################
#BLOQUE 6: Consolidación de Documentos
######################################

# Agregar importación faltante
from googleapiclient.http import MediaIoBaseDownload

def download_drive_file(drive_path, local_path, service):
    """Descarga un archivo de Drive a local"""
    try:
        folder_path, file_name = os.path.split(drive_path)
        folder_id = get_drive_folder_id(folder_path)

        if not folder_id:
            print(f"   Carpeta no encontrada: {folder_path}")
            return False

        for attempt in range(3):
            try:
                query = f"name='{file_name}' and '{folder_id}' in parents and trashed=false"
                results = service.files().list(q=query, fields="files(id)").execute()
                files = results.get('files', [])
                if not files:
                    print(f"   Archivo no encontrado: {file_name}")
                    return False
                file_id = files[0]['id']
                break
            except (HttpError, TimeoutError) as e:
                if attempt < 2:
                    time.sleep(2 ** attempt)
                else:
                    print(f"   Error buscando archivo {file_name}: {str(e)[:100]}")
                    return False

        # Descargar el archivo
        request = service.files().get_media(fileId=file_id)
        with open(local_path, 'wb') as f:
            downloader = MediaIoBaseDownload(f, request)
            done = False
            attempts = 0
            while not done and attempts < 3:
                try:
                    status, done = downloader.next_chunk()
                    attempts = 0  # reset attempts on successful chunk
                except (HttpError, TimeoutError) as e:
                    attempts += 1
                    if attempts < 3:
                        print(f"   Reintento {attempts} para {file_name}")
                        time.sleep(2 ** attempts)
                    else:
                        print(f"   Error descargando {file_name}: {str(e)[:100]}")
                        return False

        print(f"   ✓ Descargado: {file_name}")
        return True

    except Exception as e:
        print(f"   Error descargando {drive_path}: {str(e)[:200]}")
        return False

def consolidate_files(file_paths, output_name, output_dir, is_pdf=False):
    """Consolida múltiples archivos en uno solo"""
    try:
        if is_pdf:
            merger = PdfMerger()
            for path in file_paths:
                try:
                    merger.append(path)
                    print(f"      ✓ PDF añadido: {os.path.basename(path)}")
                except Exception as e:
                    print(f"      ✗ Error añadiendo PDF {os.path.basename(path)}: {str(e)[:100]}")
                    continue
            output_path = os.path.join(output_dir, output_name)
            merger.write(output_path)
            merger.close()
            return output_path
        else:
            try:
                master = Document(file_paths[0])
                composer = Composer(master)
                print(f"      ✓ Documento maestro: {os.path.basename(file_paths[0])}")

                for path in file_paths[1:]:
                    try:
                        doc = Document(path)
                        composer.append(doc)
                        print(f"      ✓ Documento añadido: {os.path.basename(path)}")
                    except Exception as e:
                        print(f"      ✗ Error añadiendo Word {os.path.basename(path)}: {str(e)[:100]}")
                        continue

                output_path = os.path.join(output_dir, output_name)
                composer.save(output_path)
                return output_path
            except Exception as e:
                print(f"      ✗ Error con el documento maestro: {str(e)[:100]}")
                return None
    except Exception as e:
        print(f"   Error consolidando {output_name}: {str(e)[:200]}")
        return None

print("Iniciando consolidación de documentos...")
service = get_drive_service()
consolidated_paths = []

# Crear carpeta base de consolidados
consolidated_base = f"{DRIVE_BASE_PATH}/Documentos_Consolidados"
get_drive_folder_id(consolidated_base)

# Procesar consolidación
print("Procesando consolidación...")

# Crear carpeta específica
selection_folder = f"{consolidated_base}/General"
selection_folder_id = get_drive_folder_id(selection_folder)

# Verificar que tenemos archivos para consolidar
print(f"Archivos Word para consolidar: {len(all_word_files)}")
print(f"Archivos PDF para consolidar: {len(all_pdf_files)}")

# Descargar y consolidar en directorio temporal
with tempfile.TemporaryDirectory() as tmpdir:
    # Procesar documentos Word
    if all_word_files:
        print(f"\n   Procesando {len(all_word_files)} archivos Word...")
        local_word_files = []

        with tqdm(total=len(all_word_files), desc="Descargando Word", unit="archivo") as pbar:
            for i, drive_path in enumerate(all_word_files):
                local_path = os.path.join(tmpdir, f"word_{i}.docx")
                if download_drive_file(drive_path, local_path, service):
                    local_word_files.append(local_path)
                pbar.update(1)

        print(f"   Archivos Word descargados exitosamente: {len(local_word_files)}/{len(all_word_files)}")

        if local_word_files:
            output_name = "Consolidado_General_Encuestas.docx"
            print(f"   Consolidando {len(local_word_files)} documentos Word...")
            consolidated_word = consolidate_files(local_word_files, output_name, tmpdir, is_pdf=False)

            if consolidated_word:
                # Subir a Drive
                try:
                    media = MediaFileUpload(consolidated_word, mimetype='application/vnd.openxmlformats-officedocument.wordprocessingml.document')
                    metadata = {'name': output_name, 'parents': [selection_folder_id]}
                    service.files().create(body=metadata, media_body=media).execute()
                    consolidated_paths.append(f"{selection_folder}/{output_name}")
                    print(f"   ✓ Word consolidado subido: {output_name}")
                except Exception as e:
                    print(f"   ✗ Error subiendo Word consolidado: {str(e)[:200]}")
            else:
                print("   ✗ No se pudo consolidar el documento Word")

    # Procesar documentos PDF
    if all_pdf_files:
        print(f"\n   Procesando {len(all_pdf_files)} archivos PDF...")
        local_pdf_files = []

        with tqdm(total=len(all_pdf_files), desc="Descargando PDF", unit="archivo") as pbar:
            for i, drive_path in enumerate(all_pdf_files):
                local_path = os.path.join(tmpdir, f"pdf_{i}.pdf")
                if download_drive_file(drive_path, local_path, service):
                    local_pdf_files.append(local_path)
                pbar.update(1)

        print(f"   Archivos PDF descargados exitosamente: {len(local_pdf_files)}/{len(all_pdf_files)}")

        if local_pdf_files:
            output_name = "Consolidado_General_Encuestas.pdf"
            print(f"   Consolidando {len(local_pdf_files)} documentos PDF...")
            consolidated_pdf = consolidate_files(local_pdf_files, output_name, tmpdir, is_pdf=True)

            if consolidated_pdf:
                # Subir a Drive
                try:
                    media = MediaFileUpload(consolidated_pdf, mimetype='application/pdf')
                    metadata = {'name': output_name, 'parents': [selection_folder_id]}
                    service.files().create(body=metadata, media_body=media).execute()
                    consolidated_paths.append(f"{selection_folder}/{output_name}")
                    print(f"   ✓ PDF consolidado subido: {output_name}")
                except Exception as e:
                    print(f"   ✗ Error subiendo PDF consolidado: {str(e)[:200]}")
            else:
                print("   ✗ No se pudo consolidar el documento PDF")

print(f"\nConsolidación completada: {len(consolidated_paths)} documentos creados")

Iniciando consolidación de documentos...
Procesando consolidación...
Archivos Word para consolidar: 6
Archivos PDF para consolidar: 6

   Procesando 6 archivos Word...


Descargando Word:   0%|          | 0/6 [00:00<?, ?archivo/s]

   ✓ Descargado: DEP217-0304.docx
   ✓ Descargado: DEC287-0605.docx
   ✓ Descargado: DEE330-0605.docx
   ✓ Descargado: DEC286-0503.docx
   ✓ Descargado: DEM223-0301.docx
   ✓ Descargado: DEE330-0602.docx
   Archivos Word descargados exitosamente: 6/6
   Consolidando 6 documentos Word...
      ✓ Documento maestro: word_0.docx
      ✓ Documento añadido: word_1.docx
      ✓ Documento añadido: word_2.docx
      ✓ Documento añadido: word_3.docx
      ✓ Documento añadido: word_4.docx
      ✓ Documento añadido: word_5.docx
   ✓ Word consolidado subido: Consolidado_General_Encuestas.docx

   Procesando 6 archivos PDF...


Descargando PDF:   0%|          | 0/6 [00:00<?, ?archivo/s]

   ✓ Descargado: DEP217-0304.pdf
   ✓ Descargado: DEC287-0605.pdf
   ✓ Descargado: DEE330-0605.pdf
   ✓ Descargado: DEC286-0503.pdf
   ✓ Descargado: DEM223-0301.pdf
   ✓ Descargado: DEE330-0602.pdf
   Archivos PDF descargados exitosamente: 6/6
   Consolidando 6 documentos PDF...
      ✓ PDF añadido: pdf_0.pdf
      ✓ PDF añadido: pdf_1.pdf
      ✓ PDF añadido: pdf_2.pdf
      ✓ PDF añadido: pdf_3.pdf
      ✓ PDF añadido: pdf_4.pdf
      ✓ PDF añadido: pdf_5.pdf
   ✓ PDF consolidado subido: Consolidado_General_Encuestas.pdf

Consolidación completada: 2 documentos creados


In [24]:
import numpy as np

print("\nExportando a Google Sheets...")

def extract_drive_id(val: str) -> str | None:
    """Extrae fileId desde un link o ID de Google Drive."""
    if not val:
        return None
    s = str(val).strip()

    if re.fullmatch(r'[A-Za-z0-9_-]{10,}', s):
        return s

    m = re.search(r'/file/d/([A-Za-z0-9_-]{10,})', s)
    if m:
        return m.group(1)

    m = re.search(r'[?&]id=([A-Za-z0-9_-]{10,})', s)
    if m:
        return m.group(1)

    m = re.search(r'open\?id=([A-Za-z0-9_-]{10,})', s)
    if m:
        return m.group(1)

    return None

def to_share_link(val: str) -> str:
    """Devuelve un link compartible (view) a partir de un ID o link."""
    fid = extract_drive_id(val)
    if not fid:
        return str(val) if val is not None else ""
    return f"https://drive.google.com/file/d/{fid}/view?usp=sharing"

def sanitize_for_sheets(df: pd.DataFrame) -> pd.DataFrame:
    """Limpia el DF para que sea JSON-compliant al subir a Sheets."""
    out = df.copy()

    for c in out.columns:
        s = out[c]
        if pd.api.types.is_datetime64_any_dtype(s):
            try:
                if getattr(s.dt, "tz", None) is not None:
                    s = s.dt.tz_convert("UTC").dt.tz_localize(None)
                else:
                    s = s.dt.tz_localize(None)
                out[c] = s.dt.strftime("%Y-%m-%d %H:%M:%S")
            except Exception:
                out[c] = s.astype(str)

    out.replace([np.inf, -np.inf], np.nan, inplace=True)

    num_cols = out.select_dtypes(include=[np.number]).columns
    if len(num_cols):
        out[num_cols] = out[num_cols].apply(lambda s: s.round(6))

    out = out.where(pd.notnull(out), "")

    for c in out.columns:
        if out[c].apply(lambda x: isinstance(x, (list, dict, set))).any():
            out[c] = out[c].astype(str)

    return out

# Preparación de datos para exportación
try:
    export_df = links_df.reset_index().rename(columns={'index': 'CURSO-HORARIO'})

    expected = ['CURSO-HORARIO', 'QR', 'Word', 'PDF']
    missing = [c for c in expected if c not in export_df.columns]
    if missing:
        raise ValueError(f"Faltan columnas en links_df: {missing}")

    export_df = export_df[expected]

    # Crear columnas con links compartibles
    export_df['QR Link']   = export_df['QR'].apply(to_share_link)
    export_df['Word Link'] = export_df['Word'].apply(to_share_link)
    export_df['PDF Link']  = export_df['PDF'].apply(to_share_link)

    # Ordenar por CURSO-HORARIO
    export_df = export_df.sort_values('CURSO-HORARIO', ascending=True)

    # Selección final de columnas
    export_df = export_df[['CURSO-HORARIO', 'QR Link', 'Word Link', 'PDF Link']]

    # Sanitizar para Sheets
    export_df = sanitize_for_sheets(export_df)

    # Escribir a Google Sheets
    try:
        worksheet = gc.open_by_key(SPREADSHEET_ID).worksheet("Enlaces_Archivos")
    except gspread.WorksheetNotFound:
        workbook = gc.open_by_key(SPREADSHEET_ID)
        worksheet = workbook.add_worksheet(title="Enlaces_Archivos", rows=1000, cols=10)

    worksheet.clear()

    header = [export_df.columns.tolist()]
    data = export_df.values.tolist()

    # Redimensionar hoja
    rows_needed = len(data) + 1
    cols_needed = len(header[0])
    try:
        worksheet.resize(rows=rows_needed, cols=max(cols_needed, 10))
    except Exception:
        pass

    # Exportación por lotes
    batch_size = 100
    total_rows = len(data)

    for i in range(0, total_rows, batch_size):
        batch = data[i:i + batch_size]
        start_row = i + 2

        if i == 0:
            worksheet.update(values=header + batch, range_name='A1')
        else:
            if start_row + len(batch) > rows_needed:
                try:
                    rows_needed = start_row + len(batch)
                    worksheet.resize(rows=rows_needed, cols=max(cols_needed, 10))
                except Exception:
                    pass

            range_start = f"A{start_row}"
            worksheet.update(values=batch, range_name=range_start)

    print("Exportación completada exitosamente")
    print(f"{len(export_df)} registros exportados")

    # Muestras
    sample_size = min(5, len(export_df))
    print("\nMuestra (primeros registros):")
    print(export_df.head(sample_size).to_string(index=False))

except Exception as e:
    print(f"Error en exportación: {e}")

print("\n¡Proceso completo de generación de QR, Word y PDF para SurveyMonkey!")
print(f"Resumen final:")
print(f"- Documentos individuales: {len(all_word_files)} Word, {len(all_pdf_files)} PDF")
print(f"- Enlaces recolectados: {len(links_df)} registros")
print(f"- Documentos consolidados: {len(consolidated_paths)}")


Exportando a Google Sheets...
Exportación completada exitosamente
6 registros exportados

Muestra (primeros registros):
CURSO-HORARIO                                                                            QR Link                                                                                                                            Word Link                                                                           PDF Link
  DEC286-0503 https://drive.google.com/file/d/1LumBmifFj6_fuQWiVFQOd4YC8w1dQfWf/view?usp=sharing https://docs.google.com/document/d/1CgbJ4gx_TD7yHCxRVpxb4QSDcT_0aRki/edit?usp=drivesdk&ouid=115555780818208664425&rtpof=true&sd=true https://drive.google.com/file/d/1izw2G4tnaZC3XjaPvek8zpd4iIdkJZQx/view?usp=sharing
  DEC287-0605 https://drive.google.com/file/d/1VQMLv6_4EgWLR1D4LbCsR_PxUSNjLRJ0/view?usp=sharing https://docs.google.com/document/d/1dCC5lpSoJ7lTbKZoiFUzOjAWYORQOa6R/edit?usp=drivesdk&ouid=115555780818208664425&rtpof=true&sd=true https://drive.google.co